In [42]:
import os
import requests
import json
import pandas as pd
from pandas import json_normalize
import time
from dotenv import load_dotenv
load_dotenv()
import openai
import whisper
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import (
                StuffDocumentsChain,
                LLMChain,
                ReduceDocumentsChain,
                MapReduceDocumentsChain,
            )
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
    create_openai_fn_runnable,
    create_structured_output_runnable,
)
from pydantic import BaseModel, Field
from typing import List, Optional
from typing import Sequence
from langchain.pydantic_v1 import BaseModel, Field
from langchain.document_loaders import TextLoader
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')

In [32]:
# Define the URL for the audio endpoint
base_url = "https://www.courtlistener.com/api/rest/v3/audio/"

# Initialize a list to store all results
all_results = []

# Loop through all pages
for page in range(11, 16):  # Adjust the range as needed
    # Construct the URL for the current page
    url = f"{base_url}?page={page}"

    # Send a GET request to the API
    response = requests.get(url)

    # Parse the JSON response
    data = json.loads(response.text)

    # Add the results from this page to the list
    all_results.extend(data["results"])
    
    # Take a nap
    time.sleep(1)

# Convert the list of results to a pandas DataFrame
df = pd.DataFrame.from_records(all_results)

In [33]:
df.shape

(100, 24)

In [34]:
pd.set_option('display.max_columns', None)

df.head()

,resource_uri,id,absolute_url,panel,docket,date_created,date_modified,source,case_name_short,case_name,case_name_full,judges,sha1,download_url,local_path_mp3,local_path_original_file,filepath_ia,ia_upload_failure_count,duration,processing_complete,date_blocked,blocked,stt_status,stt_google_response
0,https://www.courtlistener.com/api/rest/v3/audi...,88896,/audio/88896/jb-v-kyrene-elementary-school-dis...,[https://www.courtlistener.com/api/rest/v3/peo...,https://www.courtlistener.com/api/rest/v3/dock...,2023-11-08T15:31:22.569351-08:00,2023-11-10T03:42:06.317242-08:00,C,J.B.,J.B. v. Kyrene Elementary School District No. 28,,"HAWKINS, COLLINS, Murphy",c3030aa5f90925db7c247f33c40a28b0e40850b3,https://cdn.ca9.uscourts.gov/datastore/media/2...,mp3/2023/11/08/j.b._v._kyrene_elementary_schoo...,mp3/2023/11/08/j.b._v._kyrene_elementary_schoo...,https://archive.org/download/gov.uscourts.ca9....,None,1656.0,True,None,False,0,
1,https://www.courtlistener.com/api/rest/v3/audi...,88895,/audio/88895/netflix-v-babin/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-11-08T15:31:07.678313-08:00,2023-11-10T03:38:11.396431-08:00,C,Netflix,Netflix v. Babin,,,428670ffa67a76a677d5c7acdd18b43cc14bd356,https://www.ca5.uscourts.gov/OralArgRecordings...,mp3/2023/11/08/netflix_v._babin_cl.mp3,mp3/2023/11/08/netflix_v._babin.mp3,https://archive.org/download/gov.uscourts.ca5....,None,2225.0,True,None,False,0,
2,https://www.courtlistener.com/api/rest/v3/audi...,88894,/audio/88894/pointe-dallas-v-undwr-at-lloyds/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-11-08T15:30:59.382170-08:00,2023-11-09T02:02:42.963400-08:00,C,,Pointe Dallas v. Undwr at Lloyds,,,594379caefb252de019efed4988591d44a8d2acb,https://www.ca5.uscourts.gov/OralArgRecordings...,mp3/2023/11/08/pointe_dallas_v._undwr_at_lloyd...,mp3/2023/11/08/pointe_dallas_v._undwr_at_lloyd...,https://archive.org/download/gov.uscourts.ca5....,None,1448.0,True,None,False,0,
3,https://www.courtlistener.com/api/rest/v3/audi...,88893,/audio/88893/united-states-v-chiasson/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-11-08T15:30:54.643721-08:00,2023-11-09T02:07:13.803958-08:00,C,Chiasson,United States v. Chiasson,,,fc1b53568a852f849d5b56420646c81c6ea48bdc,https://www.ca5.uscourts.gov/OralArgRecordings...,mp3/2023/11/08/united_states_v._chiasson_cl.mp3,mp3/2023/11/08/united_states_v._chiasson.mp3,https://archive.org/download/gov.uscourts.ca5....,None,1707.0,True,None,False,0,
4,https://www.courtlistener.com/api/rest/v3/audi...,88892,/audio/88892/united-states-v-hinojosa/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-11-08T15:30:48.454223-08:00,2023-11-09T02:02:31.473051-08:00,C,Hinojosa,United States v. Hinojosa,,,1915553ba72c1a344f8b8f5b270e2cf2436068f8,https://www.ca5.uscourts.gov/OralArgRecordings...,mp3/2023/11/08/united_states_v._hinojosa_cl_1.mp3,mp3/2023/11/08/united_states_v._hinojosa_1.mp3,https://archive.org/download/gov.uscourts.ca5....,None,2315.0,True,None,False,0,


In [30]:
def add_rows_to_excel(input_df, excel_file):
    """
    Adds rows to an excel file. If the excel file doesn't exist, creates it. 
    If it does exist, only adds the new rows from the input DataFrame. 
    If the data overlaps, only the distinct rows are saved in the final excel file.

    Args:
    input_df (pd.DataFrame): The DataFrame to add to the excel file.
    excel_file (str): The path to the excel file.
    """
    # Convert lists to strings in input_df
    input_df = input_df.applymap(lambda x: str(x) if isinstance(x, list) else x)

    if os.path.exists(excel_file):
        # Load the existing data
        existing_df = pd.read_excel(excel_file)

        # Convert lists to strings in existing_df
        existing_df = existing_df.applymap(lambda x: str(x) if isinstance(x, list) else x)

        # Concatenate the existing data with the new data
        combined_df = pd.concat([existing_df, input_df])

        # Remove duplicates
        final_df = combined_df.drop_duplicates()

        # Print the total number of rows and how many new rows were added
        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of new rows added: {len(final_df) - len(existing_df)}")
    else:
        final_df = input_df
        # Print the total number of rows as the excel file didn't exist before
        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of new rows added: {len(final_df)}")

    # Write the data to the excel file
    final_df.to_excel(excel_file, index=False)

In [35]:
add_rows_to_excel(df, "court_listener_oral_argument_listing.xlsx")

Total number of rows: 300
Number of new rows added: 100


In [121]:
name = df.iloc[0]["case_name"]
url = df.iloc[0]["download_url"]
url

'https://cdn.ca9.uscourts.gov/datastore/media/2023/11/08/22-16816.mp3'

In [119]:
def download_mp3(url, file_path):
    """
    Downloads an MP3 file from the given URL and saves it to the specified file path.

    Args:
    url (str): The URL of the MP3 file.
    file_path (str): The local path where the file should be saved.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)

        # Raise an exception if the request was unsuccessful
        response.raise_for_status()

        # Open a file in binary write mode
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                # Write the content chunk by chunk to the file
                file.write(chunk)

        print(f"File downloaded and saved as {file_path}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

In [122]:
download_mp3(url, f"./audio/{name}.mp3")

File downloaded and saved as ./audio/J.B. v. Kyrene Elementary School District No. 28.mp3


In [123]:
model = whisper.load_model("base")
result = model.transcribe("./audio/sample_oral_arg.mp3")
print(result["text"])

 322-0440, Naperville Hotel Partners, LLC, and Superhost Hospitality Incorporated Appalines, versus Liberty Mutual Fire Insurance Company and Firemen's Fund Insurance Company Appalies. Thank you. Good afternoon. Good afternoon. Good afternoon, Your Honor. Mr. Connor, are you ready to begin? Yes, I am, Your Honor. Good afternoon. Colin Connor on behalf of the Appalines. This appeal addresses three key issues in defense motions to dismiss. First, the internal statute of limitations does not clearly, definitively, and explicitly begin to run from the beginning of the injury rather than from the end. Second, the offense should have been stopped from asserting their internal statute of limitations defense. And third, Naperville gave adequate notice of its claim. Given the high hurdles that emotion to dismiss must overcome, an Illinois law liberally constrained policies in favor of the policyholder, Naperville's reasonable interpretation of the internal statute of limitations should govern. 

In [124]:
def save_dict_to_txt_file(data_dict, file_name):
    """
    Saves the 'text' value from a dictionary to a .txt file.

    Args:
    data_dict (dict): The dictionary containing the data.
    file_name (str): The name of the .txt file to save the data to.
    """
    # Get the 'text' value from the dictionary
    text = data_dict.get('text', '')

    # Create 'transcriptions' directory if it doesn't exist
    if not os.path.exists('transcriptions'):
        os.makedirs('transcriptions')

    # Open the file in write mode
    with open(f"transcriptions/{file_name}.txt", 'w') as file:
        # Write the text to the file
        file.write(text)

    print(f"Data saved to transcriptions/{file_name}.txt")


In [125]:
def read_text_from_txt_file(file_name):
    """
    Reads the text from a .txt file.

    Args:
    file_name (str): The name of the .txt file to read the data from.
    
    Returns:
    str: The text read from the file.
    """
    # Check if the file exists
    if not os.path.exists(f"transcriptions/{file_name}.txt"):
        print(f"File transcriptions/{file_name}.txt does not exist.")
        return None

    # Open the file in read mode
    with open(f"transcriptions/{file_name}.txt", 'r') as file:
        # Read the text from the file
        text = file.read()

    return text

In [126]:
save_dict_to_txt_file(result, "lm_sample_oral_arg")

Data saved to transcriptions/lm_sample_oral_arg.txt


In [133]:
text = read_text_from_txt_file("lm_sample_oral_arg")

In [128]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000,chunk_overlap=0,separator="\n")

def run_map_reduce(input_doc, MAP_PROMPT, REDUCE_PROMPT):

    llm = ChatOpenAI(model_name="gpt-4", temperature=0.1)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)

    llm = ChatOpenAI(model_name="gpt-4", temperature=0.1)
    reduce_llm_chain = LLMChain(llm=llm, prompt=REDUCE_PROMPT)

    # Takes a list of documents and combines them into a single string
    combine_documents_chain = StuffDocumentsChain(
            llm_chain=reduce_llm_chain,
            document_variable_name="text_summaries")

    # Combines and iteravely reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `combine_documents_chain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=3500)

    # Combining documents by mapping a chain over them, then combining results
    combine_documents = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_llm_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="text",
        # Return the results of the map steps in the output
        ### Bug: this currently does not work ###
        return_intermediate_steps=False)

    # Define Map=Reduce
    map_reduce = MapReduceChain(
        # Chain to combine documents
        combine_documents_chain=combine_documents,
        # Splitter to use for initial split
        text_splitter=text_splitter)

    return map_reduce.run(input_text=input_doc)

In [129]:
map_template_string = """Context: {text}
As an expert legal analyst, review the Context which is part of an oral argument session and do three things:

1. Identify the key facts of the case.
2. Summarize plaintiff counsel's main arguments.
3. Summarize rebuttals to plaintiff counsel's arguments.
"""

reduce_template_string = template = """Context: {text_summaries}
As an experienced legal analyst, use the context to:

Compose a case brief using only the Context.
Working step by step, organize the Context into a well-structured
outline with the following sections:

Facts of Loss: a concise sumary of facts and events relevant to this legal case.
Issue: [the facts of the case being argued]
Analysis: arguments
   ["Argument 1": key points from a distinct argument,
    "Rebuttal 1": key points from the rebuttal to argument 1,
    "Argument 2": key points from a distinct argument,
    "Rebuttal 2": key points from the rebuttal to argument 2,
    "Argument X": ...,
    "Rebuttal X": ...]
Takeaway: the most important argument assertion and its corresponding rebuttal.
TODO: optional list of knowledge gaps, or logical flaws in any of the arguments or their rebuttals.
"""
MAP_PROMPT = PromptTemplate(input_variables=["text"], template=map_template_string)
REDUCE_PROMPT = PromptTemplate(input_variables=["text_summaries"], template=reduce_template_string)

In [134]:
result = run_map_reduce(text, MAP_PROMPT, REDUCE_PROMPT)

In [135]:
display(Markdown(f"{result}"))

Case Brief:

Title: Naperville Hotel Partners, LLC, and Superhost Hospitality Incorporated vs. Liberty Mutual Fire Insurance Company and Firemen's Fund Insurance Company

Facts of Loss: The case revolves around the interpretation of the internal statute of limitations in the insurance policies held by Naperville Hotel Partners, LLC, and Superhost Hospitality Incorporated with Liberty Mutual Fire Insurance Company and Firemen's Fund Insurance Company. The appellants argue that the statute of limitations should begin to run from the end of the injury, not the beginning, and that they gave adequate notice of their claim. The appellees argue that the appellants failed to file their claim within the two-year period stipulated in the policy after the occurrence of the physical damage.

Issue: The main issue is the interpretation of the internal statute of limitations in the insurance policies and whether the appellees should be estopped from asserting their internal statute of limitations defense.

Analysis:
Argument 1: The appellants argue that the internal statute of limitations does not clearly, definitively, and explicitly begin to run from the beginning of the injury rather than from the end. They also argue that the appellees should have been estopped from asserting their internal statute of limitations defense.
Rebuttal 1: The appellees argue that the appellants' interpretation of the internal statute of limitations is not reasonable and would lead to absurd results. They also argue that the appellants' claim that the appellees should be estopped from asserting their internal statute of limitations defense is not valid.

Argument 2: The appellants contend that they gave adequate notice of their claim and that the insurers' policies do not include any words like "starts" and "steps" or "begins" in their policy.
Rebuttal 2: The appellees contend that the appellants knew about the water damage and should have filed their claim within the two-year period stipulated in the policy. They also dispute the claim that the insurers continued to investigate the claim after the statute of limitations had expired.

Takeaway: The most important argument assertion is the interpretation of the internal statute of limitations in the insurance policies and its corresponding rebuttal is that the appellants' interpretation is not reasonable and would lead to absurd results.

Knowledge Gaps: The case does not provide information on the specific wording of the insurance policies, which could have a significant impact on the interpretation of the internal statute of limitations. Additionally, the case does not provide information on the specific timeline of events, which could affect the validity of the appellants' claim that they gave adequate notice of their claim.

In [136]:
class CourtroomParty(BaseModel):
    """Identifying information about a person."""
    
    person: str = Field(..., description="The name of the person speaking")
    role: Optional[str] = Field(None, description="The role of the person speaking")
    background: Optional[str] = Field(None, description="A concise summary about the person")
    

class OralArgument(BaseModel):
    """Identifying information about a particular argument."""
    
    topic: str = Field(..., description="A concise topic title for the argument")
    side: Optional[str] = Field(None, description="The side the argument favors, plaintiff or defense")
    person: Optional[str] = Field(None, description="The name of the person presenting the argument")
    summary: Optional[str] = Field(None, description="A concise summary of key points")
    

class OralDocket(BaseModel):
    """Identifying information in a text."""

    parties: Sequence[CourtroomParty] = Field(..., description="The information of interest in the text")
    arguments: Sequence[OralArgument] = Field(..., description="The information of interest in the text")

In [137]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 0,
    is_separator_regex = False,
)

In [138]:
loader = TextLoader("transcriptions/lm_sample_oral_arg.txt")
doc = loader.load()

In [139]:
docs = text_splitter.split_documents(doc)

In [140]:
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world class algorithm for extracting information in structured formats.",
        ),
        (
            "human",
            "Use the given format to extract information from the following input: {input}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

In [141]:
runnable = create_structured_output_runnable(OralDocket, llm, prompt)

In [142]:
res_docs = []

for i in range(len(docs)):
    res_doc = runnable.invoke({"input": docs[i]})
    res_docs.append(res_doc)

In [143]:
# Convert each object instance to a dictionary
data_dicts = [instance.dict() for instance in res_docs]

# Write the list of dictionaries to a JSON file
with open('data.json', 'w') as f:
    json.dump(data_dicts, f)

In [144]:
# Create a DataFrame for parties
parties_data = [data_dict['parties'] for data_dict in data_dicts]
df_parties = pd.concat([json_normalize(data) for data in parties_data], ignore_index=True)

# Create a DataFrame for arguments
arguments_data = [data_dict['arguments'] for data_dict in data_dicts]
df_arguments = pd.concat([json_normalize(data) for data in arguments_data], ignore_index=True)

In [151]:
df_parties.to_excel("example_case_parties.xlsx", index=False)
df_arguments.to_excel("example_case_topics.xlsx", index=False)

In [145]:
print(f"Shape of parties df: {df_parties.shape}")
df_parties

Shape of parties df: (42, 3)


,person,role,background
0,Colin Connor,Counsel,Representative for the Appalines
1,"Naperville Hotel Partners, LLC",Plaintiff,One of the Appalines in the case
2,Superhost Hospitality Incorporated,Plaintiff,One of the Appalines in the case
3,Liberty Mutual Fire Insurance Company,Defendant,One of the Appalies in the case
4,Firemen's Fund Insurance Company,Defendant,One of the Appalies in the case
5,Strauss,Plaintiff,Involved in a case regarding the timing of fil...
6,Pinar Amicase Court,Court,Analyzed the issue of policy terms regarding t...
7,Midwest Dairy Products,Referenced Entity,Illinois Supreme Court case cited for the prin...
8,Defendant's agent,Agent,Listed the period of exposure on the loss noti...
9,Naperville,Plaintiff,Filed an action before the statute of limitati...


In [146]:
df_parties.role.value_counts()

role
Attorney                8
Plaintiff               7
Defense                 4
Counsel                 3
Defendant               3
Judge                   2
Speaker                 2
Unknown                 2
Case Reference          2
Referenced Entity       1
Agent                   1
Legal Representative    1
Court                   1
Lawyer                  1
Justice                 1
Policy Holder           1
Insurance Company       1
Witness                 1
Name: count, dtype: int64

In [147]:
print(f"Shape of arguments df: {df_arguments.shape}")
df_arguments

Shape of arguments df: (59, 4)


,topic,side,person,summary
0,Statute of Limitations,Plaintiff,Colin Connor,The internal statute of limitations should beg...
1,Estoppel of Limitations Defense,Plaintiff,Colin Connor,The defense should be estopped from asserting ...
2,Adequate Notice of Claim,Plaintiff,Colin Connor,Naperville gave adequate notice of its claim.
3,Interpretation of Policy Language,Plaintiff,Colin Connor,"Illinois law favors policyholders, and the rea..."
4,Policy Language on Physical Loss or Damage,Plaintiff,Colin Connor,The insurers' policies do not clearly state wh...
5,Contractual Limitation Period,Defense,Strauss,The policy required that suit be filed within ...
6,Policy Language Interpretation,Plaintiff,Pinar Amicase Court,After a loss occurs is fundamentally different...
7,Insurance Benefit Protection,Plaintiff,Midwest Dairy Products,A policyholder should not be deprived of the b...
8,Statute of Limitations,Plaintiff,Naperville,"The damage occurred until at least April 30, 2..."
9,Policy Language and Claim Assertion,Plaintiff,Counsel,"According to the policy language, you get to w..."


In [148]:
df_arguments.person.value_counts()

person
Unknown                   13
Naperville                12
Colin Connor               5
Howard Fishman             4
Council                    3
Mr. Conner                 3
Mr. Counter                3
Mr. Cheryl                 3
Strauss                    1
Pinar Amicase Court        1
Midwest Dairy Products     1
Counsel                    1
Defendants                 1
Your Honor                 1
Mr. Fishman                1
Name: count, dtype: int64

In [149]:
for i in range(len(res_docs)):
    for j in range(len(res_docs[i].parties)):
        display(Markdown(f"{res_docs[i].parties[j]}"))

person='Colin Connor' role='Counsel' background='Representative for the Appalines'

person='Naperville Hotel Partners, LLC' role='Plaintiff' background='One of the Appalines in the case'

person='Superhost Hospitality Incorporated' role='Plaintiff' background='One of the Appalines in the case'

person='Liberty Mutual Fire Insurance Company' role='Defendant' background='One of the Appalies in the case'

person="Firemen's Fund Insurance Company" role='Defendant' background='One of the Appalies in the case'

...

person='Mr. Conner' role='Counsel' background='References an internal statute of limitations and argues for a contractual provision as a condition preceding to coverage.'

person='Mr. Fishman' role='Attorney' background="Presented arguments for affirming the trial court's decision."

person='Mr. Conner' role='Attorney' background='Presented rebuttal arguments focusing on policy language and reasonableness.'

person='Unknown' role='Counsel' background='Arguing on behalf of the policyholder against Liberty Mutual regarding the statute of limitations and notice provisions in an insurance claim.'

person='Mr. Connor' role='Witness' background=None

In [150]:
for i in range(len(res_docs)):
    for j in range(len(res_docs[i].arguments)):
        display(Markdown(f"{res_docs[i].arguments[j]}"))

topic='Statute of Limitations' side='Plaintiff' person='Colin Connor' summary="The internal statute of limitations should begin from the end of the injury, not the beginning, and Naperville's interpretation should govern."

topic='Estoppel of Limitations Defense' side='Plaintiff' person='Colin Connor' summary='The defense should be estopped from asserting their internal statute of limitations defense.'

topic='Adequate Notice of Claim' side='Plaintiff' person='Colin Connor' summary='Naperville gave adequate notice of its claim.'

topic='Interpretation of Policy Language' side='Plaintiff' person='Colin Connor' summary='Illinois law favors policyholders, and the reasonable interpretation of the policy language by Naperville regarding the statute of limitations should be accepted.'

topic='Policy Language on Physical Loss or Damage' side='Plaintiff' person='Colin Connor' summary="The insurers' policies do not clearly state when the limitation period starts, and precedent suggests it runs from the conclusion of the loss."

...

topic='Reservation of Rights Letter' side='Plaintiff' person=None summary='Liberty Mutual failed to include the statute of limitations defense in their Reservation of Rights Letter, despite listing other policy defenses.'

topic='Investigation Post-Statute of Limitations' side='Plaintiff' person=None summary='Liberty Mutual continued to investigate the claim and implied coverage even after the statute of limitations had allegedly expired.'

topic='41-Month Delay' side='Plaintiff' person=None summary='Clarifies that there was no 41-month delay in providing notice as the cumulative effect of the loss was not determined until May 2019.'

topic='Conclusion of Questioning' side='Defense' person='Unknown' summary='No further questions for Mr. Connor, acknowledgment of the end of the session for the day.'

topic='Deliberation' side='Court' person='Unknown' summary='The court thanks the participants for their arguments and will take the matter under advisement, with a written decision to follow.'

In [153]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings)

retriever = db.as_retriever()

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [154]:
result = chain.invoke("Generate a well strucured list of legal arguments.")

In [155]:
Markdown(f"{result}")

Based on the provided context, here is a structured list of legal arguments that were likely presented in the case:

1. Waiver Argument:
   - The insurance company argues that they are not obligated to repeatedly set out the suit limitation provision in every reservation of rights letter or communication with the insured.
   - They cite the Foam Craft case from Illinois, which states that the insured cannot blame the insurance company for failing to read the policy and discover the requirements for bringing suit.
   - The insurance company points out that there is no Illinois law or regulation, other than section 919.80 of the Illinois Administrative Code, that requires them to specify the suit limitation provision when denying coverage.

2. Estoppel Argument:
   - The insurance company refutes the claim that their conduct during the investigation and request for additional documents should estop them from enforcing the suit limitation provision.
   - They clarify that during the investigation in September, they were still unaware of when the suit limitation had expired.

3. Reasonableness and Policy Language:
   - The opposing party argues that the considerations should focus on the policy language drafted by the insurers and the reasonableness of the insured's actions.
   - They reference out-of-state cases (Panorama and Strauss) that interpreted similar policy language and found that the statute of limitations did not begin to run until the conclusion of the loss.
   - They emphasize that these cases were decided before the insurance policies in question were written, suggesting that the insurers should have been aware of these interpretations.

4. Continuous Loss Argument:
   - The opposing party argues that the specific dates of loss listed in the complaint were only examples and that water intrusion damage is a continuous process that may not immediately lead to damage.
   - They suggest that the insurance policy should cover the ongoing damage until the loss concludes.

5. Policy Interpretation and Construction:
   - The insurance company argues that the insurance policy must be construed as a whole, giving effect to every provision, citing Supreme Court cases LaVore and Central Illinois Light.
   - They claim that the interpretation urged by the opposing party is unreasonable and would lead to absurd results, allowing the insured to delay notification indefinitely.

6. LaVore C Factors for Reasonable Notice:
   - The insurance company argues that the 41-month delay in notification is unreasonable.
   - They point to the insured's sophistication in commerce and insurance, as evidenced by their hiring of an insurance coverage attorney.
   - They assert that the insured was aware of the event triggering coverage due to visible remediation efforts.
   - They challenge the opposing party's "actionable occurrence" test for diligence as unsupported by Illinois law or policy language, referencing the Breckenridge case.

These arguments reflect the legal positions taken by the parties in the case as inferred from the provided context.